# Chapter 18. with, match, and else Blocks

`with` statement sets up a temporary context and reliably tears it down, under the control of a context manager object.

Context manager object is the result of evaluating expression after `with`, but the value bound to the target variable (in `as` clause) is the result returned by `__enter__` method of the context manager object.

In [ ]:
# open('mirror.py') returns in instance of `TextIOWrapper`
# whose __enter__ method returns self

# some context manager's __enter__ returns None i.e. `as` is optional
with open('mirror.py') as fp:
  src = fp.read(60)
  # when control flow exist the with block, the __exit__ method is invoked on the context manager object (not )

In [ ]:
len(src)

60

In [ ]:
src

'def just_random_text(str):\n  return f"{str} hahaha! Hello!"\n'

In [ ]:
fp

<_io.TextIOWrapper name='mirror.py' mode='r' encoding='UTF-8'>

In [ ]:
fp.closed, fp.encoding

(True, 'UTF-8')

In [ ]:
fp.read(60)

ValueError: I/O operation on closed file.

In [ ]:
# mirror.py
import sys

class LookingGlass:
  def __enter__(self): # Python invokes __enter__ with no arguments other than self
    self.original_write = sys.stdout.write # hold the original `sys.stdout.write`
    sys.stdout.write = self.reverse_write
    return 'JABBERWOCKY'

  def reverse_write(self, text):
    self.original_write(text[::-1])

  # If all went well, python calls it with None, None, None
  # Otherwise, it'll hold information about exception
  def __exit__(self, exc_type, exc_value, traceback):
    sys.stdout.write = self.original_write
    if exc_type is ZeroDivisionError:
      print("Please DO NOT divide by zero!")
      return True # tell interpreter that everything is handled
                  # Otherwise, any exception raised in with block will be propagated


In [ ]:
with LookingGlass() as what:
  print('Alice, Kitty and Snowdrop')
  print(what)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ


In [ ]:
what

'JABBERWOCKY'

In [ ]:
print('Back to normal')

Back to normal


In [ ]:
manager = LookingGlass()
manager

In [ ]:
monster = manager.__enter__()

In [ ]:
monster == "JABBERWOCKY"

True

In [ ]:
monster

'JABBERWOCKY'

In [ ]:
manager

In [ ]:
manager.__exit__(None, None, None)

In [ ]:
monster

'JABBERWOCKY'

In [ ]:
sys.stdout.write("hi"[::-1])

ih

In [ ]:
import sys

original_write = None

def reverse_write(text):
  original_write(text[::-1])

def enter(): # Python invokes __enter__ with no arguments other than self
  original_write = sys.stdout.write # hold the original `sys.stdout.write`
  sys.stdout.write = reverse_write
  return original_write



In [ ]:
original_write = enter()

In [ ]:
print("hi")

ih


In [ ]:
print("yeah")

haey


In [ ]:
sys.stdout.write = original_write

In [ ]:
print("oh")

oh


### The contextlib Utilities

Most widely used of these utilities is the `@contextmanager` decorator. This is also intersting because it shows a use for the `yield` statement unrelated to iteration.

### Using `@contextmanager`
 - brings three distinctive Python features: a function decorator, a generator, and the `with` statement
 - instead of writing `__enter__` and `__exit__`, you just implement a generator with a single `yield` that should produce whatever you want the `__enter__` method to return.

In [ ]:
# mirror_gen.py

import contextlib
import sys

@contextlib.contextmanager
def looking_glass():
  original_write = sys.stdout.write

  def reverse_write(text):
    original_write(text[::-1])

  sys.stdout.write = reverse_write # everything before yield will be executed
                                  #  at the beinning of the with block

  msg = ''
  try:
    yield 'JABBERWOCKY'
  except ZeroDivisionError:
    msg = "Please DO NOT divide by zero!"
  finally:
    sys.stdout.write = original_write
    if msg:
      print(msg)

In [ ]:
with looking_glass() as what:
  a = 1.0
  b = 0.0
  a / b
  print("Alice, Kitty and Snowdrop")
  print(what)

Please DO NOT divide by zero!


In [ ]:
print("hi")

hi


In [ ]:
@looking_glass()
def verse():
  print("The time has come")

In [ ]:
verse()

emoc sah emit ehT


In [ ]:
print('back to the normal')

back to the normal


The problem with the following code:
 1. The `fileinput` has an API that heavily relies on glbals

In [2]:
# Bad example
import fileinput

for line in fileinput.input("randomtext.txt", inplace=True):
  line = 'additional information ' + line.rstrip('\n')
  print(line)

In [4]:
import os
os.extsep

'.'

In [7]:
# using @contextlib.contextmanager
# it focuses on just one file / ignores sys.stdin
from contextlib import contextmanager
import io
import os

@contextmanager
def inplace(filename, mode='r', buffering=-1, encoding=None, errors=None,
            newline=None, backup_extension=None):
  """
  Allow for a file to be replaced with new content
  """
  if set(mode).intersection('wa+'):
    raise ValueError('Only read-only file modes can be used')

  backupfilename = filename + (backup_extension or os.extsep + 'bak')
  try:
    os.unlink(backupfilename) # delete the existing backup file
  except os.error:
    pass

  os.rename(filename, backupfilename)
  readable = io.open(backupfilename, mode, buffering=buffering,
                     encoding=encoding, errors=errors, newline=newline)

  try:
    perm = os.fstat(readable.fileno()).st_mode
  except OSError:
    writable = open(filename, 'w' + mode.replace('r', ''),
                    buffering=buffering, encoding=encoding, errors=errors,
                    newline=newline)
  else:
    os_mode = os.O_CREAT | os.O_WRONLY | os.O_TRUNC
    if hasattr(os, 'O_BINARY'):
      os_mode |= os.O_BINARY
    fd = os.open(filename, os_mode, perm)
    writable = io.open(fd, 'w' + mode.replace('r', ''), buffering=buffering,
                       encoding=encoding, errors=errors, newline=newline)

    try:
      if hasattr(os, 'chmod'):
        os.chmod(filename, perm)
    except OSError:
      pass
  # Everything before `yield` deals with
  # setting up the context, which entails
  # creating a backup file,
  # then opening and yielding references to the readable
  # and writable file handles that will be returned by the
  # __enter__ call
  try:
    yield readable, writable
  except Exception:
    # move backupback
    try:
      os.unlink(filename)
    except os.error:
      pass
    os.rename(backupfilename, filename)
    raise
  finally:
    readable.close()
    writable.close()
    try:
      os.unlink(backupfilename)
    except os.error:
      pass


In [8]:
import csv
csvfilename = "random.csv"

with inplace(csvfilename, 'r', newline='') as (infh, outfh):
  reader = csv.reader(infh)
  writer = csv.writer(outfh)

  for row in reader:
    row += ['new', 'columns']
    writer.writerow(row)

## Pattern Matching in lis.py: A Case Study

Why Norvig's `lis.py`?
 1. It's a beautiful example of idiomatic Python code
 2. simplicity of Scheme is a master class of language design
 3. learning how an interpreter works can give us a deeper understanding of Python and prog languages

In [9]:
import math
import operator as op
from collections import ChainMap
from itertools import chain
from typing import Any, TypeAlias, NoReturn

Symbol: TypeAlias = str # just an alias for str
                        # In this codoe, it's used for identifiers
Atom: TypeAlias = float | int | Symbol # a simple syntatic elem
                                       # number or str
Expression: TypeAlias = Atom | list # building block of scheme programs

In [17]:
def parse(program: str) -> Expression:
  "Read a Scheme expression from a string"
  return read_from_tokens(tokenize(program))

def tokenize(s: str) -> list[str]:
  # convert a string into a list of tokens
  return s.replace('(', ' ( ').replace(')', ' ) ').split()


def read_from_tokens(tokens: list[str]) -> Expression:
  "read an expression from a sequence of tokens."
  if len(tokens) == 0:
    raise SyntaxError('unexpected EOF while reading')
  token = tokens.pop(0)
  if '(' == token:
    exp = []
    while tokens[0] != ')':
      exp.append(read_from_tokens(tokens))
    tokens.pop(0) # discard ')'
    return exp
  elif ')' == token:
    raise SyntaxError('unexpected )')
  else:
    return parse_atom(token)

def parse_atom(token: str) -> Atom:
  try:
    return int(token)
  except ValueError:
    try:
      return float(token)
    except ValueError:
      return Symbol(token)

In [19]:
parse('(mod m n)')

['mod', 'm', 'n']

In [20]:
parse('1.5')

1.5

In [21]:
parse('ni!')

'ni!'

In [22]:
parse('(gcd 18 45)')

['gcd', 18, 45]

In [23]:
parse('''
(define double
  (lambda (n)
    (* n 2)))
''')

['define', 'double', ['lambda', ['n'], ['*', 'n', 2]]]

Using the terminology of the Python interpreter, the output of `parse` is an AST (Abstract Syntax Tree): a convenient representation of the Scheme program as nested lists forming a tree-like structure, where the outermost list is the trunk, inner lists are the branches, and atoms are the leaves

In [24]:
parse('(lambda (a b) (* (/ a b) 100))')

['lambda', ['a', 'b'], ['*', ['/', 'a', 'b'], 100]]

### The Environment

In [25]:
class Environment(ChainMap[Symbol, Any]):
  "A ChainMap that allows changing an item in-place"

  def change(self, key: Symbol, value: Any) -> None:
    "Find where key is defined and change the value"
    for map in self.maps:
      if key in map:
        map[key] = value # type: ignore[index]
        return
    raise KeyError(key)

In [26]:
inner_env = {'a': 2}
outer_env = {'a': 0 , 'b': 1}
env = Environment(inner_env, outer_env)

In [27]:
env['a']

2

In [28]:
env['b']

1

In [29]:
env['a'] = 111

In [31]:
env['c'] = 222

In [32]:
env

Environment({'a': 111, 'c': 222}, {'a': 0, 'b': 1})

In [33]:
env.change('b', 333)

In [34]:
env

Environment({'a': 111, 'c': 222}, {'a': 0, 'b': 333})

In [36]:
def standard_env() -> Environment:
  "it builds and returns teh global environment"
  env = Environment()
  env.update(vars(math)) # sin, cos, sqrt, pi, ..
  env.update({
      '+': op.add,
      '-': op.sub,
      '*': op.mul,
      '/': op.truediv,
      'quotient': op.floordiv,
      '>': op.gt,
      '<': op.lt,
      '>=': op.ge,
      '<=': op.le,
      '=': op.eq,
      'abs': abs,
      'append': lambda *args: list(chain(*args)),
      'apply': lambda proc, args: proc(*args),
      'begin': lambda *x: x[-1],
      'car': lambda x: x[0],
      'cdr': lambda x: x[1:],
      'cons': lambda x, y: [x] + y,
      'display': lambda x: print(lispstr(x)),
      'eq?': op.is_,
      'equal?': op.eq,
      'filter': lambda *args: list(filter(*args)),
      'length': len,
      'list': lambda *x: list(x),
      'list?': lambda x: isinstance(x, list),
      'map': lambda *args: list(map(*args)),
      'max': max,
      'min': min,
      'not': op.not_,
      'null?': lambda x: x == [],
      'number?': lambda x: isinstance(x, (int, float)),
      'procedure?': callable,
      'round': round,
      'symbol?': lambda x: isinstance(x, Symbol),
  })
  return env

To summarize, the `env` mapping is loaded with:
 - All fcns from Python's `math` module
 - Selected operators from Python's `op` module
 - Simple but powerful fcns built with Python's `lambda`
 - Python built-ins renamed, like `callable` as `procedue?`

### The REPL
REPL (Read-Eval-Print-Loop) is easy to understand but not user-friendly.

In [ ]:
def repl(prompt: str = 'lis.py> ') -> NoReturn:
  "A prompt-read-eval-prit loop"
  global_env = Environment({}, standard_env())
  while True:
    ast = parse(input(prompt))
    val = evaluate(ast, global_env)
    if val is not None:
      print(lispstr(val))

def lispstr(exp: object) -> str:
  "Convert a Python obj back into a Lisp-readable string"
  if isinstance(exp, list):
    return '(' + ' '.join(map(lispstr, exp)) + ')'
  else:
    return str(exp)